<a href="https://colab.research.google.com/github/sanjaypriyadarsan/Emotion-Detection-from-Uploaded-Images/blob/main/Facial_emotion.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
!ls "/content/drive/MyDrive/archive.zip"


/content/drive/MyDrive/archive.zip


In [ ]:
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

# Define transforms
transform = transforms.Compose([
    transforms.Resize((48, 48)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(10),
    transforms.Grayscale(num_output_channels=3),
    transforms.ToTensor()
])


In [ ]:
# Unzip directly from Google Drive to Colab
!unzip -q "/content/drive/MyDrive/archive.zip" -d "/content/dataset"

# Paths to folders (make sure your zip has "train" and "test" folders inside)
train_path = "/content/dataset/train"
test_path = "/content/dataset/test"

from torchvision import datasets
from torch.utils.data import DataLoader

# Load with ImageFolder
train_dataset = datasets.ImageFolder(root=train_path, transform=transform)
test_dataset = datasets.ImageFolder(root=test_path, transform=transform)

# Create DataLoaders
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

print("Classes:", train_dataset.classes)


Classes: ['angry', 'disgust', 'fear', 'happy', 'neutral', 'sad', 'surprise']


In [ ]:
import torch.nn as nn
import torch.nn.functional as F

class EmotionCNN(nn.Module):
    def __init__(self):
        super(EmotionCNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.pool = nn.MaxPool2d(2, 2)
        self.dropout = nn.Dropout(0.25)
        self.fc1 = nn.Linear(64 * 12 * 12, 128)
        self.fc2 = nn.Linear(128, 7)  # 7 emotion classes

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))  # -> [32x24x24]
        x = self.pool(F.relu(self.conv2(x)))  # -> [64x12x12]
        x = x.view(-1, 64 * 12 * 12)
        x = self.dropout(F.relu(self.fc1(x)))
        x = self.fc2(x)
        return x


In [ ]:
import torch
from torch import optim

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = EmotionCNN().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Training Loop
epochs = 20
for epoch in range(epochs):
    model.train()
    running_loss = 0.0

    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    print(f"Epoch [{epoch+1}/{epochs}], Loss: {running_loss:.4f}")


Epoch [1/20], Loss: 764.2971
Epoch [2/20], Loss: 685.9161
Epoch [3/20], Loss: 652.5140
Epoch [4/20], Loss: 630.8722
Epoch [5/20], Loss: 614.3741
Epoch [6/20], Loss: 601.9964
Epoch [7/20], Loss: 590.6472
Epoch [8/20], Loss: 581.8261
Epoch [9/20], Loss: 575.2051
Epoch [10/20], Loss: 567.1076
Epoch [11/20], Loss: 563.3552
Epoch [12/20], Loss: 557.1713
Epoch [13/20], Loss: 550.5250
Epoch [14/20], Loss: 545.1949
Epoch [15/20], Loss: 539.3688
Epoch [16/20], Loss: 537.8284
Epoch [17/20], Loss: 532.0634
Epoch [18/20], Loss: 527.7833
Epoch [19/20], Loss: 526.8604
Epoch [20/20], Loss: 520.8660


In [ ]:
from sklearn.metrics import classification_report, confusion_matrix
import numpy as np

model.eval()
all_preds = []
all_labels = []

with torch.no_grad():
    for images, labels in test_loader:
        images = images.to(device)
        labels = labels.to(device)

        outputs = model(images)
        _, preds = torch.max(outputs, 1)

        all_preds.extend(preds.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())

In [ ]:
print("Classification Report:\n")
print(classification_report(all_labels, all_preds, target_names=train_dataset.classes))


Classification Report:

              precision    recall  f1-score   support

       angry       0.45      0.36      0.40       958
     disgust       0.56      0.16      0.25       111
        fear       0.49      0.18      0.26      1024
       happy       0.67      0.81      0.73      1774
     neutral       0.45      0.56      0.50      1233
         sad       0.39      0.47      0.43      1247
    surprise       0.70      0.69      0.69       831

    accuracy                           0.53      7178
   macro avg       0.53      0.46      0.47      7178
weighted avg       0.53      0.53      0.52      7178



In [ ]:
torch.save(model.state_dict(), "emotion_cnn.pth")


In [ ]:
from google.colab import files
files.download("emotion_cnn.pth")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
!pip install streamlit

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.0/10.0 MB 37.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 55.2 MB/s eta 0:00:00


In [ ]:
!pip install pyngrok

In [ ]:
# model.py
%%writefile model.py
import torch.nn as nn
import torch.nn.functional as F

class EmotionCNN(nn.Module):
    def __init__(self):
        super(EmotionCNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, 3, padding=1)
        self.conv2 = nn.Conv2d(32, 64, 3, padding=1)
        self.pool = nn.MaxPool2d(2, 2)
        self.dropout = nn.Dropout(0.25)
        self.fc1 = nn.Linear(64 * 12 * 12, 128)
        self.fc2 = nn.Linear(128, 7)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 64 * 12 * 12)
        x = self.dropout(F.relu(self.fc1(x)))
        x = self.fc2(x)
        return x


Writing model.py


In [ ]:
# app.py
%%writefile app.py
import streamlit as st
import torch
from torchvision import transforms
from PIL import Image
import cv2
import numpy as np
import mediapipe as mp
from model import EmotionCNN

# Load the model
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = EmotionCNN().to(device)
model.load_state_dict(torch.load('emotion_cnn.pth', map_location=device))
model.eval()

# Emotion labels
EMOTIONS = ['angry', 'disgust', 'fear', 'happy', 'neutral', 'sad', 'surprise']

# Preprocessing for prediction
transform = transforms.Compose([
    transforms.Resize((48, 48)),
    transforms.Grayscale(num_output_channels=3),
    transforms.ToTensor()
])

# Mediapipe face detection
mp_face_detection = mp.solutions.face_detection
face_detector = mp_face_detection.FaceDetection(model_selection=0, min_detection_confidence=0.5)

# Predict emotion
def predict_emotion(face_img):
    face_img = Image.fromarray(face_img)
    face_tensor = transform(face_img).unsqueeze(0).to(device)
    with torch.no_grad():
        output = model(face_tensor)
        _, predicted = torch.max(output, 1)
    return EMOTIONS[predicted.item()]

# App UI
st.title("🧠 Emotion Detection from Image")
st.write("Upload an image, and I’ll detect the face and predict the emotion!")

uploaded_file = st.file_uploader("Upload a JPG/PNG image", type=["jpg", "jpeg", "png"])

if uploaded_file is not None:
    image = Image.open(uploaded_file).convert('RGB')
    img_array = np.array(image)
    img_rgb = cv2.cvtColor(img_array, cv2.COLOR_RGB2BGR)

    # Face detection
    results = face_detector.process(cv2.cvtColor(img_rgb, cv2.COLOR_BGR2RGB))

    if results.detections:
        for det in results.detections:
            bboxC = det.location_data.relative_bounding_box
            h, w, _ = img_rgb.shape
            x, y, w_box, h_box = int(bboxC.xmin * w), int(bboxC.ymin * h), int(bboxC.width * w), int(bboxC.height * h)
            face = img_rgb[y:y + h_box, x:x + w_box]

            try:
                emotion = predict_emotion(face)
                st.image(img_rgb, caption="Detected Face", channels="BGR")
                st.success(f"Predicted Emotion: **{emotion.upper()}**")
            except:
                st.warning("Couldn't process face properly. Try another image.")
    else:
        st.warning("No face detected. Please upload a clearer image.")


Writing app.py


In [ ]:
!pip install mediapipe

In [ ]:
!pip install easyocr

In [ ]:
!npm install localtunnel

⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋
added 22 packages in 4s
⠋
⠋3 packages are looking for funding
⠋  run `npm fund` for details
⠋

In [ ]:
!streamlit run /content/app.py &>/content/logs.txt & npx localtunnel --port 8501 & curl ipv4.icanhazip.com

34.82.230.55
⠙your url is: https://lucky-phones-kick.loca.lt


In [ ]:
streamlit.__version__


'1.49.1'

In [ ]:
import streamlit

TypeError: Descriptors cannot be created directly.
If this call came from a _pb2.py file, your generated code is out of date and must be regenerated with protoc >= 3.19.0.
If you cannot immediately regenerate your protos, some other possible workarounds are:
 1. Downgrade the protobuf package to 3.20.x or lower.
 2. Set PROTOCOL_BUFFERS_PYTHON_IMPLEMENTATION=python (but this will use pure-Python parsing and will be much slower).

More information: https://developers.google.com/protocol-buffers/docs/news/2022-05-06#python-updates

In [ ]:
!pip install streamlit==1.4.0

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.3/9.3 MB 51.4 MB/s eta 0:00:00
  Attempting uninstall: streamlit
    Found existing installation: streamlit 1.2.0
    Uninstalling streamlit-1.2.0:
      Successfully uninstalled streamlit-1.2.0


In [ ]:
%%writefile app1.py
import streamlit as st
uploaded_file = st.file_uploader("Upload a JPG/PNG image", type=["jpg", "jpeg", "png"])
st.write(uploaded_file)

Writing app1.py
